In this Kaggle kernel, we will perform an ablation study to find the connection between model size and validation accuracy with the dataset that I have created in this [Kaggle kernel](https://www.kaggle.com/ayuraj/rainforest-create-image-dataset-with-w-b). 

We will set up a [Weights and Biases Sweep](https://docs.wandb.com/sweeps) to find the best model for our dataset. We will also see the effect of batch size and learning rate along with the model size. 

# Imports and Setups

In [ ]:
!nvidia-smi

In [ ]:
SEED = 666

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
tf.random.set_seed(SEED)

from tensorflow import keras
from tensorflow.keras.datasets import cifar10

In [ ]:
import os
import csv
os.environ["TF_DETERMINISTIC_OPS"] = "1"

import numpy as np
np.random.seed(SEED)

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

%matplotlib inline

from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import librosa as lb 
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from skimage.transform import resize
from scipy import stats

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

# Download Dataset from W&B Artifacts and Prepare

In [ ]:
run = wandb.init(project='rainforest', job_type='download_dataset')

artifact = run.use_artifact('wandb/rainforest/spectrogram-dataset_nfft_2024_hop_512:v0', type='dataset')
artifact_dir = artifact.download()

run.join()

In [ ]:
IMG_DIR = Path(artifact_dir+'/')
IMG_PATH = list(map(str, list(IMG_DIR.glob('*.bmp'))))

### Train-test Split

In [ ]:
train_path, valid_path = train_test_split(IMG_PATH, test_size=0.20, shuffle=True, random_state=42)
len(IMG_PATH), len(train_path), len(valid_path)

### Dataloader

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
IMG_WIDTH = 400
IMG_HEIGHT = 224
CHANNELS = 3
NUM_CLASSES = 24

In [ ]:
@tf.function
def parse_data(image_path):
    # parse image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    # normalize image
    image = tf.image.per_image_standardization(image)
    
    # parse data
    label = tf.strings.split(image_path, sep='_')[-2]
    label = tf.strings.to_number(label, out_type=tf.int32)
    label = tf.one_hot(label, NUM_CLASSES) 
    
    return image, label

# Model

In [ ]:
def get_model(base_model_name):
    if base_model_name=='resnet':
        base_model = tf.keras.applications.ResNet50V2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
        
    elif base_model_name=='efficientnetb0':
        base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    elif base_model_name=='efficientnetb1':
        base_model = tf.keras.applications.EfficientNetB1(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    elif base_model_name=='efficientnetb2':
        base_model = tf.keras.applications.EfficientNetB2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    
    elif base_model_name=='xception':
        base_model = tf.keras.applications.Xception(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    
    elif base_model_name=='densenet':
        base_model = tf.keras.applications.DenseNet169(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

    base_model.trainabe = True  
    
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, 3))
    resize = experimental.preprocessing.Resizing(224,224)(inputs) 
    x = base_model(resize, training=True)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  
    outputs = Dense(NUM_CLASSES, activation='sigmoid')(x)
    
    return Model(inputs, outputs)

# Setup Sweep 

In [ ]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'base_model_name': 'resnet',
        'dataset_name': 'nfft_2024_hop_512'
        }

    # Initialize wandb with a sample project name
    run = wandb.init(config=config_defaults)

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 30

    # download and prepare data
    data_artifact = run.use_artifact('wandb/rainforest/spectrogram-dataset_{}:latest'.format(wandb.config.dataset_name))
    artifact_dir = data_artifact.download()
    
    IMG_DIR = Path(artifact_dir+'/')
    IMG_PATH = list(map(str, list(IMG_DIR.glob('*.bmp'))))
    
    # train-validation split
    train_path, valid_path = train_test_split(IMG_PATH, test_size=0.20, shuffle=True, random_state=42)
    
    # dataloader
    trainloader = tf.data.Dataset.list_files((train_path))
    validloader = tf.data.Dataset.list_files((valid_path))

    trainloader = (
        trainloader
        .shuffle(1024)
        .map(parse_data, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    validloader = (
        validloader
        .map(parse_data, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )


    # Iniialize model with hyperparameters
    keras.backend.clear_session()
    model = get_model(wandb.config.base_model_name)
    
    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(opt, 'binary_crossentropy', metrics=['acc'])
    
    # Train the model
    _ = model.fit(trainloader,
                  epochs=wandb.config.epochs, 
                  validation_data=validloader,
                  callbacks=[WandbCallback()]) # WandbCallback to automatically track metrics
                            
    # Evaluate    
    loss, accuracy = model.evaluate(validloader, callbacks=[WandbCallback()])
    print('Test Error Rate: ', round((1-accuracy)*100, 2))
    wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)}) # wandb.log to track custom metrics
    
    # save model
    model.save('model.h5')

    # initialize a new artifact to save the model
    model_artifact =  wandb.Artifact("trained-model", 
                                     type="model", 
                                     description="Simple model trained with spectrogram dataset formed with nfft 2024 and hop length of 512",
                                     metadata={'optimizer': 'Adam',
                                              'Loss': 'Binary Cross Entropy',
                                              'Learning Rate': 0.001})

    model_artifact.add_file('model.h5')
    run.log_artifact(model_artifact)
    
    run.join()

# Sweep Config

In [ ]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'max_iter': 27,
      's': 2,
      'eta': 3
  },
  'parameters': {
      'base_model_name': {
          'values': ['resnet',
                     'efficientnetb0',
                     'efficientnetb1',
                     'efficientnetb2',
                     'xception',
                     'densenet']
      },
      'dataset_name': {
          'values': ['nfft_2024_hop_512',
                     'nfft_1024_hop_512',
                     'nfft_1024_hop_256']
      }
  }
}

# Initialize Sweep and Run Agent

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="rainforest")

In [ ]:
wandb.agent(sweep_id, function=train)